<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Ensemble_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble Methods
 Ensemble methods is a machine learning technique that combines several base models in order to produce one optimal predictive model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception

Create two models, here we have taken Inception_V3 and Xception.
Then take their layer on which you want output

In [ ]:
incep_base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(250,250,3))
incep_base_model.trainable = False
incep_base_model_out =  incep_base_model.get_layer("mixed10").output

In [ ]:
xcep_base_model = Xception(weights="imagenet", include_top=False, input_shape=(250,250,3))
xcep_base_model.trainable = False
xcep_base_model_out = xcep_base_model.get_layer("block14_sepconv2_act").output

Then, create two diiferent models and comiple them and train them on your database.

In [ ]:
x = incep_base_model_out
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
output = tf.keras.layers.Dense(100, activation="softmax")(x)

incep_model = tf.keras.Model(incep_base_model.input, output)
incep_model.summary()

In [ ]:
x = xcep_base_model_out
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
output = tf.keras.layers.Dense(100, activation="softmax")(x)

xcep_model = tf.keras.Model(xcep_base_model.input, output)
xcep_model.summary()

In [ ]:
incep_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
xcep_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

In [ ]:
incep_model.fit(train_df, validation_data=val_df, epochs=10, callbacks=[tensorboard,checkpoint_incep])


In [ ]:
xcep_model.fit(train_df, validation_data=val_df, epochs=10, callbacks=[tensorboard,checkpoint])

In [ ]:
Then, take the two models and combine them to get the best result. Sometimes you get the best result, but sometime

In [ ]:
# Ensemble model

input = tf.keras.Input(shape=(None,None,3))

y_1 = incep_model(input)
y_2 = xcep_model(input)

output = 0.5*y_1 + 0.5*y_2
ensemble_model = tf.keras.Model(input, output)

Use below to predict and plot the images



In [ ]:
testing_dir = os.path.join("/content/testing/")
list_test = os.listdir(testing_dir)
plt.figure(figsize=(10,10), constrained_layout=True)
for a,x in enumerate(list_test):
  image = load_img(testing_dir+x, target_size=(250,250))
  img = tf.keras.utils.img_to_array(image)
  img = normalize_layer(img)
  img = tf.expand_dims(img, axis=0)
  pred = ensemble_model.predict(img)
  game_name = class_name[np.argmax(pred)]
  plt.subplot(4,4,a+1)
  plt.imshow(image)
  plt.title(game_name)